In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00


In [ ]:
import torch
import os
import pandas as pd
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, GPT2Config
from torch.utils.data import DataLoader, Dataset

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Running on", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available. Running on CPU.")

GPU is available. Running on Tesla V100-SXM2-16GB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tuning_T = pd.read_excel('/content/drive/MyDrive/딥러닝 챗봇/0728 이후/T값 엑셀.xlsx')

In [ ]:
print(f'tuning_T의 데이터 크기는 {len(tuning_T)}')

tuning_T의 데이터 크기는 1928


In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

# 모델을 학습 시킬 때 negative_df에서는 486개 / positive_df에서는 1806개를 가져온다

In [ ]:
class QADataset(Dataset):
    def __init__(self, questions, answers, tokenizer):
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = self._find_max_length()
    def _find_max_length(self):
        max_length = 0
        for question, answer in zip(self.questions, self.answers):
            input_text = f"{question} {self.tokenizer.eos_token} {answer} {self.tokenizer.eos_token}"
            inputs = self.tokenizer(input_text, truncation=True, return_tensors="pt")
            max_length = max(max_length, inputs.input_ids.size(1))
        return max_length
    def __len__(self):
        return len(self.questions)
    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]
        # 질문과 답변을 결합하여 입력값과 라벨로 만듦
        input_text = f"{question} {self.tokenizer.eos_token} {answer} {self.tokenizer.eos_token}"
        inputs = self.tokenizer(input_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        labels = inputs.input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100
        return {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': labels.squeeze()
        }

In [ ]:
def train_model(model, tokenizer, train_dataset, num_epochs=10, batch_size=2, lr=2e-5, device='cuda', model_save_path=None):
    # 모델 로드 및 기존 파라미터 고정

    model.to(device)
    model.train()

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{num_epochs} - Avg. Loss: {avg_loss:.4f}")

    return model

In [ ]:
questions = tuning_T['Speaker_1'].values.tolist()
answers = tuning_T['Speaker_2'].values.tolist()

In [ ]:
answers

['무슨 일 있어?',
 '몇시?',
 '너 하고싶은거 있어?',
 '감기는 면역력이 떨어졌을때 걸리기 쉬우니까 몸관리를 잘해봐',
 '그래 걱정해줘서 고마워',
 '내가 말을 어떻게 했는데?',
 '아 미안해!',
 '무슨 차 샀음?',
 '홍차ㅋㅋ',
 '기억남',
 '뭐 좀 먹어',
 '그래야겠다 진짜 배고파ㅋㅋ',
 '뭐 먹을건데?',
 '치킨 먹을까 했어 ',
 '어디꺼 먹을까? 좋아하는 브랜드 있어?',
 '좋아 맛있게 먹어!',
 '굿굿',
 '완전 좋지',
 '그래 난 좋아!',
 '왜 싸웠는데?',
 '왜 걔가 뭐 잘못함?',
 '의견차이지 뭐',
 '다시 대화를 해봐야겠다',
 '오래 끌지 말고 바로 연락해봐',
 '그래 ',
 '맞아 오늘 34도래',
 '나도ㅋㅋ 근데 가을 엄청 짧아서..',
 '안찝찝하고 좋지 ',
 '예산 얼마 있는데?',
 '20만원정도',
 '나이가들거나 아프면 죽겠지',
 '너무 깊이 생각하지마ㅠ ',
 '뉴스 보다가 생각났어',
 '지각할거같다고 생각하면 지각맞음',
 'ㅋㅋㅋ화이팅',
 '교환해야겠다',
 '그상황이라면 최대한 빨리 도망가는게 맞을듯',
 '맞아.. 진짜 안타깝더라 ',
 '없어 왜?',
 '오키 뭐하고?',
 'ㅇㅋ',
 '뭐하나 궁금했어',
 '걍 계속 눈감고있으면 잠ㅋ',
 '병원가보셈ㅋ',
 '지금?',
 '그래',
 '탈모약 먹으셈 ㅋㅋ 근데 부작용 좀있다더라',
 '언제?',
 '갔다와! 어디바다 갈거야?',
 '그럼 인천 고?',
 '어떻게 썼길래?',
 '그럴때냐 할거먼저해라',
 '아 그럼 견뎌야지ㅋㅋㅋ 뭐삼?',
 '오 어디꺼',
 '아 그거 예쁘더라 ',
 '당연하지 ',
 '헐 어떻게생겼어?',
 '운동을 시키던가 사료를 바꿔야할듯',
 '그럼 어쩔수 없지ㅋㅋ',
 '귀여우면 된거야',
 '왜?',
 '여름은 너무 더워',
 '아 인정 ',
 'ㅋㅋ겉옷 들고다니는 수밖에 없어',
 '너무 오래쓰면 그런듯 바꿀때 되면 바꿔야지 뭐',
 '강사님 쓰는거 써봐',
 '몰라 ㅋㅋㅋ

In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')


train_dataset = QADataset(questions, answers, tokenizer)

fine_tuned_model = train_model(model, tokenizer, train_dataset, num_epochs=100, batch_size=2, lr=2e-5, device='cuda')


torch.save(fine_tuned_model.state_dict(), "1929_Final_xlsxT.pth")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Epoch 1/100 - Avg. Loss: 3.4885
Epoch 2/100 - Avg. Loss: 2.5544
Epoch 3/100 - Avg. Loss: 1.9084
Epoch 4/100 - Avg. Loss: 1.4026
Epoch 5/100 - Avg. Loss: 1.0466
Epoch 6/100 - Avg. Loss: 0.8000
Epoch 7/100 - Avg. Loss: 0.6507
Epoch 8/100 - Avg. Loss: 0.5612
Epoch 9/100 - Avg. Loss: 0.5076
Epoch 10/100 - Avg. Loss: 0.4497
Epoch 11/100 - Avg. Loss: 0.4214
Epoch 12/100 - Avg. Loss: 0.4011
Epoch 13/100 - Avg. Loss: 0.3743
Epoch 14/100 - Avg. Loss: 0.3607
Epoch 15/100 - Avg. Loss: 0.3374
Epoch 16/100 - Avg. Loss: 0.3272
Epoch 17/100 - Avg. Loss: 0.3253
Epoch 18/100 - Avg. Loss: 0.3022
Epoch 19/100 - Avg. Loss: 0.3015
Epoch 20/100 - Avg. Loss: 0.2789
Epoch 21/100 - Avg. Loss: 0.2773
Epoch 22/100 - Avg. Loss: 0.2798
Epoch 23/100 - Avg. Loss: 0.2617
Epoch 24/100 - Avg. Loss: 0.2543
Epoch 25/100 - Avg. Loss: 0.2563
Epoch 26/100 - Avg. Loss: 0.2434
Epoch 27/100 - Avg. Loss: 0.2457
Epoch 28/100 - Avg. Loss: 0.2452
Epoch 29/100 - Avg. Loss: 0.2382
Epoch 30/100 - Avg. Loss: 0.2308
Epoch 31/100 - Avg.

In [ ]:
# train_dataset = QADataset(questions, answers, tokenizer)

# # 모델 학습
# trained_model = train_model(model, tokenizer, train_dataset, num_epochs=10)

In [ ]:
# torch.save(trained_model.state_dict(), "positive4500negative500.h5")

In [ ]:
# class QADataset(Dataset):
#     def __init__(self, questions, answers, tokenizer, max_length=128):
#         self.questions = questions
#         self.answers = answers
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.questions)

#     def __getitem__(self, idx):
#         question = self.questions[idx]
#         answer = self.answers[idx]

#         # 질문과 답변을 결합하여 입력값과 라벨로 만듦
#         input_text = f"{question} {self.tokenizer.eos_token} {answer}"
#         inputs = self.tokenizer(input_text, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
#         labels = inputs.input_ids.clone()
#         labels[labels == self.tokenizer.pad_token_id] = -100

#         # 토큰화된 입력 데이터를 리스트 형태로 반환
#         return {'input_ids': inputs.input_ids[0],
#                 'attention_mask': inputs.attention_mask[0],
#                 'labels': labels[0]}

# def test_model(model, tokenizer, test_dataset, batch_size=32, device='cuda'):
#     model.to(device)
#     model.eval()

#     test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

#     all_predictions = []

#     with torch.no_grad():
#         for batch in test_dataloader:
#             # 토크나이저를 사용하여 입력 데이터를 토큰화
#             inputs = {k: v.to(device) for k, v in batch.items()}  # 수정된 부분
#             outputs = model.generate(
#                 input_ids=inputs['input_ids'],
#                 attention_mask=inputs['attention_mask'],
#                 max_length=150,  # 텍스트가 길 수 있으므로 적절한 max_length 설정
#                 pad_token_id=tokenizer.pad_token_id,
#                 bos_token_id=tokenizer.bos_token_id,
#                 eos_token_id=tokenizer.eos_token_id,
#                 num_beams=5,  # Beam Search 크기
#                 # num_beams=1,
#                 early_stopping=True  # 최대 길이에 도달하면 멈춤
#             )
#             all_predictions.extend(outputs.tolist())

#     # 예측 결과를 텍스트로 디코딩하여 반환
#     decoded_predictions = [tokenizer.decode(prediction, skip_special_tokens=True) for prediction in all_predictions]

#     return decoded_predictions

class QADataset(Dataset):
    def __init__(self, questions, answers, tokenizer):
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = self._find_max_length()
    def _find_max_length(self):
        max_length = 0
        for question, answer in zip(self.questions, self.answers):
            input_text = f"{question} {self.tokenizer.eos_token} {answer}"
            inputs = self.tokenizer(input_text, truncation=True, return_tensors="pt")
            max_length = max(max_length, inputs.input_ids.size(1))
        return max_length
    def __len__(self):
        return len(self.questions)
    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]

        # 답변을 토큰화하여 입력값과 라벨로 만듦
        answer_input = f"{answer} {self.tokenizer.eos_token}"
        answer_inputs = self.tokenizer(answer_input, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
        answer_labels = answer_inputs.input_ids.clone()
        answer_labels[answer_labels == self.tokenizer.pad_token_id] = -100

        # 토큰화된 입력 데이터를 리스트 형태로 반환
        return {'answer_input_ids': answer_inputs.input_ids[0],
                'answer_attention_mask': answer_inputs.attention_mask[0],
                'answer_labels': answer_labels[0]}

def test_model(model, tokenizer, test_dataset, batch_size=32, device='cuda'):
    model.to(device)
    model.eval()

    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    all_predictions = []

    with torch.no_grad():
        for batch in test_dataloader:
            # 토크나이저를 사용하여 답변만 토큰화
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model.generate(
                input_ids=inputs['answer_input_ids'],
                attention_mask=inputs['answer_attention_mask'],
                max_length=150,
                pad_token_id=tokenizer.pad_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                num_beams=5,
                early_stopping=True
            )
            all_predictions.extend(outputs.tolist())

    # 예측 결과를 텍스트로 디코딩하여 반환
    decoded_predictions = [tokenizer.decode(prediction, skip_special_tokens=True) for prediction in all_predictions]

    return decoded_predictions

In [ ]:
test_questions = [
    '요즘 나 힘들어',
    '오늘 날씨 어때?',
    '날씨가 너무 추워',
    '자고 일어나서 뭐할꺼야',
    '오늘 뭐 먹었어?',
    '지금 뭐해?',
    '야구 좋아해?',
    '치킨 브랜드 추천',
    '치킨 어느 치킨 먹었어?',
    '밥 뭐먹을까?',
    '콜라하고 사이다 둘 중 어느것이 좋아?',
    '언제 잘까?',
    '제일 유명한 축구 팀?',
    '스타벅스에서 제일 좋아하는 것이 뭐야?',
    '너 지금 뭐해',
    'ㅎㅇ?',
    '모해',
    '😡',
    '😁',
    '🤬',
    '😉',
    '😍',
    '😛',
    '🥱',
    '😉 장난이야'
]

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# 테스트 데이터셋 준비
test_dataset = QADataset(test_questions, test_questions, tokenizer)

# 학습된 모델을 불러옴
model.load_state_dict(torch.load("/content/1929_Final_xlsxT.pth"))

# 모델 테스트
predictions = test_model(model, tokenizer, test_dataset)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [ ]:
for question, prediction in zip(test_questions, predictions):
    print(f"질문: {question}")
    print(f"예측 답변: {prediction}\n")

질문: 요즘 나 힘들어
예측 답변: 요즘 나 힘들어  들어간  

질문: 오늘 날씨 어때?
예측 답변: 오늘 날씨 어때?  뭐 좋은 날씨 좋네 

질문: 날씨가 너무 추워
예측 답변: 날씨가 너무 추워  ᄋᄋ 얇게 입어 

질문: 자고 일어나서 뭐할꺼야
예측 답변: 자고 일어나서 뭐할꺼야? 

질문: 오늘 뭐 먹었어?
예측 답변: 오늘 뭐 먹었어?  뭐 먹었어 

질문: 지금 뭐해?
예측 답변: 지금 뭐해?  뭐해 

질문: 야구 좋아해?
예측 답변: 야구 좋아해?  하입보이 

질문: 치킨 브랜드 추천
예측 답변: 치킨 브랜드 추천  먹자 

질문: 치킨 어느 치킨 먹었어?
예측 답변: 치킨 어느 치킨 먹었어?  먹었어  

질문: 밥 뭐먹을까?
예측 답변: 밥 뭐먹을까?  먹는거 어때? 

질문: 콜라하고 사이다 둘 중 어느것이 좋아?
예측 답변: 콜라하고 사이다 둘 중 어느것이 좋아?  봐 

질문: 언제 잘까?
예측 답변: 언제 잘까?  괜찮을지? 

질문: 제일 유명한 축구 팀?
예측 답변: 제일 유명한 축구 팀?  믿어? 

질문: 스타벅스에서 제일 좋아하는 것이 뭐야?
예측 답변: 스타벅스에서 제일 좋아하는 것이 뭐야? 크비싸다 ᄏ 

질문: 너 지금 뭐해
예측 답변: 너 지금 뭐해  누구랑 대화 

질문: ㅎㅇ?
예측 답변: ᄒᄋ? ᄋᄋ 

질문: 모해
예측 답변: 모해 니터 추천좀ᄏ 

질문: 😡
예측 답변:  화났어? 

질문: 😁
예측 답변:  왜 웃어? 

질문: 🤬
예측 답변:  좀 화나네 

질문: 😉
예측 답변:  장난이야 

질문: 😍
예측 답변:  뭔데? 

질문: 😛
예측 답변:  메롱 

질문: 🥱
예측 답변:  뭘 보고 있어? 

질문: 😉 장난이야
예측 답변: 장난이야  장난이야 

